# 0. Imports

In [59]:
import numpy as np

# 1. Environment Config

In [ ]:
env_conf = {
    "GRID_SIZE" : 4,
    "GAMMA" : 0.9,
    "GOAL_STATE" : 3,
    "THRESHOLD" : 0.00001
}

# 2. Create Environment

In [61]:
# Extract all configs into variables
GRID_SIZE, GAMMA, GOAL_STATE, THRESHOLD = env_conf['GRID_SIZE'],  env_conf['GAMMA'], env_conf['GOAL_STATE'], env_conf['THRESHOLD']
NUM_STATES = GRID_SIZE * GRID_SIZE

In [62]:
# Create an reward map with the specifications
rewards = np.zeros(NUM_STATES)
rewards[GOAL_STATE] = 1

In [63]:
# Create an action map, i.e. where does a specified action take you from state X to Y as index changes
actions = {
    "up" : -GRID_SIZE,
    "down" : GRID_SIZE,
    "left" : -1,
    "right": 1
}

In [64]:
# Initialise a Value matrix
V = np.zeros(NUM_STATES)

# 3. Supplementary Function

In [65]:
def action_validation(state: int, action: str):
    """
    Function: Validates if an action can be taken by the agent in that state
    Args:
        state(int) : Grid position of the agent
        action(str) :  Action it wants to take
    Returns:
        Bool: True for valid action and False for invalid actions
    """

    # Check the row, column
    row, col = divmod(state, GRID_SIZE)

    # For VERTICAL BOUNDS
    if (row == 0 and action == "up") or (row == GRID_SIZE - 1 and action == "down"):
        return False
    
    # For HORIZONTAL BOUNDS
    if (col == 0 and action == "left") or (col == GRID_SIZE -1 and action == "right"):
        return False
    
    # Else return true
    return True
    

In [66]:
def get_next_state(state, action):
    """
    Function: Validates the action and the state annd produces the next state
    Args:
        state (int): Position index in the grid worrld
        action (str): The action the agent intends to take 
    Returns:
        next_state (int) : The next state index
    """

    # Check if the action is valid, if not return the same state back
    if not action_validation(state=state, action=action):
        return state
    
    else:
        return state + actions[action]


In [67]:
def print_grid(V: np.array, GRID_SIZE: int):
    """
    Function: Prints the gridfrom the value matrix and the provided GRID_SIZE
    Args:
        V (np.array): Value matrix with the value of each state V(s)
        GRID_SIZE (int): The size of one of the GRID DIMENSIOONs of the square grid
    """

    print(np.round(V.reshape((GRID_SIZE , GRID_SIZE)),2 ), '\n')


In [68]:
def extract_policy(V: np.array):
    """
    Function: Prints the optimal policy from the Value matrix
    Args:
        V (np.array): The value matrix
    """
    policy_arrows = []
    for state in range(NUM_STATES):
        if state == GOAL_STATE:
            policy_arrows.append("G")
            continue

        best_action = None
        best_value = float('-inf')

        for action in actions:
            if not action_validation(state, action):
                continue
            next_state = get_next_state(state, action)
            r = rewards[next_state]
            value = r + GAMMA * V[next_state]
            if value > best_value:
                best_value = value
                best_action = action

        arrow = {
            'up': '↑',
            'down': '↓',
            'left': '←',
            'right': '→'
        }.get(best_action, '?')

        policy_arrows.append(arrow)

    # Print policy grid
    print("Optimal Policy:")
    policy_grid = np.array(policy_arrows).reshape(GRID_SIZE, GRID_SIZE)
    for row in policy_grid:
        print("  ".join(row))

# 4. Value Iteration

In [ ]:
def value_iteration(V: np.matrix, NUM_STATES: int, GOAL_STATE: int, GRID_SIZE: int):
    """
    Function: Recurrsively iterates through the Value grid till the updates are less than THRESHOLD
    Args:
        V (np.matrix): The value grid
        NUM_STATES (int): Number of states in the grid world
        GOAL_STATE (int): The goal state index
        GRID_SIZE (int): Dimensions of one of the sides of the grid
    Returns: The value iterated Value Matrix
    """
    
    # Define the iteration tracker
    iteration = 0

    while True:

        # Define the value confs
        delta = 0
        new_V = np.copy(V)
        iteration += 1

        
        # Loop through and update the value of each state
        for state in range(NUM_STATES):

            # Skip updating the GOAL STATES value
            if state == GOAL_STATE:
                continue

            # Explore each action in the state 
            # Initialise the max value as =inf 
            # Track the maximum possible value in each state
            max_value = - np.inf
            for action in actions.keys():
                next_state = get_next_state(state=state, action=action)
                r = rewards[next_state]
                value = r + GAMMA * V[next_state]
                max_value = max(max_value, value)

            # Update the value as the max iteration 
            new_V[state] = max_value
            delta = max(delta, abs(V[state] - new_V[state]))

        # Update the value grid
        V = new_V
        print(f"Iteration {iteration}")

        print_grid(V=V, GRID_SIZE=GRID_SIZE)

        # Break if the delta is too low
        if delta < THRESHOLD:
            break

    return V

# SANDBOX

In [70]:
V = value_iteration(V=V, 
                    NUM_STATES=NUM_STATES, 
                    GOAL_STATE=GOAL_STATE, 
                    GRID_SIZE=GRID_SIZE)

Iteration 1
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 

Iteration 2
[[0.  0.9 1.  0. ]
 [0.  0.  0.9 1. ]
 [0.  0.  0.  0.9]
 [0.  0.  0.  0. ]] 

Iteration 3
[[0.81 0.9  1.   0.  ]
 [0.   0.81 0.9  1.  ]
 [0.   0.   0.81 0.9 ]
 [0.   0.   0.   0.81]] 

Iteration 4
[[0.81 0.9  1.   0.  ]
 [0.73 0.81 0.9  1.  ]
 [0.   0.73 0.81 0.9 ]
 [0.   0.   0.73 0.81]] 

Iteration 5
[[0.81 0.9  1.   0.  ]
 [0.73 0.81 0.9  1.  ]
 [0.66 0.73 0.81 0.9 ]
 [0.   0.66 0.73 0.81]] 

Iteration 6
[[0.81 0.9  1.   0.  ]
 [0.73 0.81 0.9  1.  ]
 [0.66 0.73 0.81 0.9 ]
 [0.59 0.66 0.73 0.81]] 

Iteration 7
[[0.81 0.9  1.   0.  ]
 [0.73 0.81 0.9  1.  ]
 [0.66 0.73 0.81 0.9 ]
 [0.59 0.66 0.73 0.81]] 



In [71]:
extract_policy(V)

Optimal Policy:
→  →  →  G
↑  ↑  ↑  ↑
↑  ↑  ↑  ↑
↑  ↑  ↑  ↑
